In [1]:
!pip install textgen

     |████████████████████████████████| 92kB 5.3MB/s 
     |████████████████████████████████| 2.5MB 12.5MB/s 
     |████████████████████████████████| 245kB 45.0MB/s 
     |████████████████████████████████| 23.9MB 123kB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 122kB 44.2MB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 3.3MB 34.9MB/s 
     |████████████████████████████████| 901kB 39.6MB/s 
     |████████████████████████████████| 245kB 50.0MB/s 
     |████████████████████████████████| 122kB 46.0MB/s 
  Created wheel for textgen: filename=textgen-0.0.4-cp37-none-any.whl size=103589 sha256=3a50897a5a3dc0a73c7f63e395b7a18db3f8eec795f1bcd2fa87cb3e9af98618
  Stored in directory: /root/.cache/pip/wheels/64/50/b0/ada9ec5cc3b33650774eca922ba2fe590cc67b0dd32e3f4401
  Created wheel for text2vec: filename=text2vec-0.1.9-cp37-none-any.whl size=77264 sha256=b31b6e0a8a7390ade8858d73f25648bf0f871f

In [2]:
import torch
import sqlite3
import pandas as pd
from textgen.seq2seq import Seq2SeqModel
from sklearn.model_selection import train_test_split

conn = sqlite3.connect('/content/songs.db')
c = conn.cursor()

In [3]:
df = pd.read_sql('SELECT * FROM Splitted', conn)
df1 = df[['genre','full_lyrics', 'full_chorus', 'full_verses']].copy()
X_train, X_test, y_train, y_test = train_test_split(df1, df['title'], stratify = df1['genre'], test_size=0.15, random_state=42) 

In [4]:
use_cuda = torch.cuda.is_available()
model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 8,
    "num_train_epochs": 5,
    "save_eval_checkpoints": False,
    "save_model_every_epoch": False,
    "silent": False,
    "evaluate_generated_text": True,
    "evaluate_during_training": False,
    "evaluate_during_training_verbose": False,
    "use_multiprocessing": False,
    "save_best_model": True,
    "output_dir": './en_output/',
}

In [5]:
def count_matches(labels, preds):
    print(labels)
    print(preds)
    return sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])


def title_results(col):
  train_df = pd.DataFrame(list(zip(X_train[col].values, y_train)), columns=["input_text", "target_text"])
  eval_df = pd.DataFrame(list(zip(X_test[col].values, y_test)), columns=["input_text", "target_text"])
  model = Seq2SeqModel("bert", "bert-base-cased", "bert-base-cased", args=model_args, use_cuda=use_cuda, )
  model.train_model(train_df, eval_data=eval_df, matches=count_matches)
  with torch.no_grad():
    print(model.eval_model(eval_df, matches=count_matches))

In [6]:
title_results('full_lyrics')

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationshi

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

[   INFO 20210624 07:15:39 seq2seq_utils: 239]  Saving features into cached file cache_dir/bert-base-cased-bert-base-cased_cached_128591


Running Evaluation:   0%|          | 0/74 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/74 [00:00<?, ?it/s]

[   INFO 20210624 07:16:12 seq2seq_model: 918] {'eval_loss': 4.926094799428372, 'matches': 0}


['Jungle', '\u200bconey island', 'Chemtrails Over the Country Club', 'Juice', 'Mouth of the River', 'All of the Lights', 'Cat’s in the Cradle', 'My Number', 'This Love', 'Everybody', 'BTS - Pied Piper (English Translation)', "Walt Grace's Submarine Test, January 1967", 'Nina Cried Power', '\u200b\u200bparty favor', 'Stay', 'Present Tense', 'Love You Goodbye', 'Spirits', 'Are You What You Want to Be?', 'Anaconda', 'Bad', 'Sparkle (English Version)', '2002', 'Forbidden Fruit', 'Those Were the Days', "I Don't Love You", '\u200btraitor', 'Valerie', 'BTS - 작은 것들을 위한 시 (Boy With Luv) ft. Halsey (English Translation)', 'Motion Sickness', 'Soul Searching', '\u200bpick up the phone', 'Seventeen', 'Cradles', 'Turn Out the Lights', 'Cold Water', 'Fade', 'Finally // beautiful stranger', '\u200bgetting it on', 'BUTTERFLY EFFECT', 'No Guidance', 'Paradise', 'Emmylou', 'ME!', 'FourFiveSeconds', 'Hey There Delilah', 'Victorious', 'XO', 'New Flesh', "No Tellin'", 'Stolen Dance', 'How Deep Is Your Love'

In [7]:
title_results('full_chorus')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationshi

  0%|          | 0/3344 [00:00<?, ?it/s]

[   INFO 20210624 07:16:23 seq2seq_utils: 239]  Saving features into cached file cache_dir/bert-base-cased-bert-base-cased_cached_1283344
[   INFO 20210624 07:16:24 seq2seq_model: 566]  Training started


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/418 [00:00<?, ?it/s]

...

[   INFO 20210624 07:28:31 seq2seq_utils: 239]  Saving features into cached file cache_dir/bert-base-cased-bert-base-cased_cached_128591


Running Evaluation:   0%|          | 0/74 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/74 [00:00<?, ?it/s]

[   INFO 20210624 07:29:02 seq2seq_model: 918] {'eval_loss': 4.769532691788029, 'matches': 0}


['Jungle', '\u200bconey island', 'Chemtrails Over the Country Club', 'Juice', 'Mouth of the River', 'All of the Lights', 'Cat’s in the Cradle', 'My Number', 'This Love', 'Everybody', 'BTS - Pied Piper (English Translation)', "Walt Grace's Submarine Test, January 1967", 'Nina Cried Power', '\u200b\u200bparty favor', 'Stay', 'Present Tense', 'Love You Goodbye', 'Spirits', 'Are You What You Want to Be?', 'Anaconda', 'Bad', 'Sparkle (English Version)', '2002', 'Forbidden Fruit', 'Those Were the Days', "I Don't Love You", '\u200btraitor', 'Valerie', 'BTS - 작은 것들을 위한 시 (Boy With Luv) ft. Halsey (English Translation)', 'Motion Sickness', 'Soul Searching', '\u200bpick up the phone', 'Seventeen', 'Cradles', 'Turn Out the Lights', 'Cold Water', 'Fade', 'Finally // beautiful stranger', '\u200bgetting it on', 'BUTTERFLY EFFECT', 'No Guidance', 'Paradise', 'Emmylou', 'ME!', 'FourFiveSeconds', 'Hey There Delilah', 'Victorious', 'XO', 'New Flesh', "No Tellin'", 'Stolen Dance', 'How Deep Is Your Love'

In [9]:
title_results('full_verses')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationshi

  0%|          | 0/3344 [00:00<?, ?it/s]

[   INFO 20210624 07:29:41 seq2seq_utils: 239]  Saving features into cached file cache_dir/bert-base-cased-bert-base-cased_cached_1283344
[   INFO 20210624 07:29:41 seq2seq_model: 566]  Training started


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/418 [00:00<?, ?it/s]

...

[   INFO 20210624 07:41:50 seq2seq_utils: 239]  Saving features into cached file cache_dir/bert-base-cased-bert-base-cased_cached_128591


Running Evaluation:   0%|          | 0/74 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/74 [00:00<?, ?it/s]

[   INFO 20210624 07:42:21 seq2seq_model: 918] {'eval_loss': 5.02852467266289, 'matches': 0}


['Jungle', '\u200bconey island', 'Chemtrails Over the Country Club', 'Juice', 'Mouth of the River', 'All of the Lights', 'Cat’s in the Cradle', 'My Number', 'This Love', 'Everybody', 'BTS - Pied Piper (English Translation)', "Walt Grace's Submarine Test, January 1967", 'Nina Cried Power', '\u200b\u200bparty favor', 'Stay', 'Present Tense', 'Love You Goodbye', 'Spirits', 'Are You What You Want to Be?', 'Anaconda', 'Bad', 'Sparkle (English Version)', '2002', 'Forbidden Fruit', 'Those Were the Days', "I Don't Love You", '\u200btraitor', 'Valerie', 'BTS - 작은 것들을 위한 시 (Boy With Luv) ft. Halsey (English Translation)', 'Motion Sickness', 'Soul Searching', '\u200bpick up the phone', 'Seventeen', 'Cradles', 'Turn Out the Lights', 'Cold Water', 'Fade', 'Finally // beautiful stranger', '\u200bgetting it on', 'BUTTERFLY EFFECT', 'No Guidance', 'Paradise', 'Emmylou', 'ME!', 'FourFiveSeconds', 'Hey There Delilah', 'Victorious', 'XO', 'New Flesh', "No Tellin'", 'Stolen Dance', 'How Deep Is Your Love'